In [ ]:
from picamera import PiCamera
import time
import numpy as np
from tensorflow.lite.python.interpreter import Interpreter
from PIL import Image
import os

# Define the path to the saved TFLite model
model_path = 'D:/EDI Dataset/trained_model.tflite'

# Load the TFLite model
interpreter = Interpreter(model_path=model_path)
interpreter.allocate_tensors()

# Define a dictionary to map the class indices to their labels
label_map = {0: '10', 1: '20', 2: '50', 3: '100', 4: '200', 5: '500', 6: '2000'}

# Initialize the PiCamera
camera = PiCamera()

# Define the path to store the captured image
capture_path = '/home/ayush/Desktop/photos'

# Create a unique filename for the captured image
filename = 'image.jpg'

# Capture an image and save it to the capture directory
camera.capture(os.path.join(capture_path, filename))

# Wait for a short time
time.sleep(1)

# Define a list to store the predicted labels
predictions = []

# Loop over the captured images
for filename in os.listdir(capture_path):
    # Load the image
    img = Image.open(os.path.join(capture_path, filename))
    # Preprocess the image
    input_tensor = interpreter.get_input_details()[0]['index']
    input_shape = interpreter.get_input_details()[0]['shape']
    resized_image = img.resize((input_shape[1], input_shape[2]))
    input_data = np.array(resized_image).astype('float32')
    input_data = np.expand_dims(input_data, axis=0)
    input_data /= 255.0
    # Set the input tensor
    interpreter.set_tensor(input_tensor, input_data)
    # Perform inference
    interpreter.invoke()
    # Get the output tensor
    output_tensor = interpreter.get_output_details()[0]['index']
    output_data = interpreter.get_tensor(output_tensor)
    # Get the predicted class index
    predicted_class_index = np.argmax(output_data)
    # Get the predicted class label from the label_map dictionary
    predicted_label = label_map[predicted_class_index]
    # Add the predicted label to the predictions list
    predictions.append(predicted_label)

# Print the predictions
print(predictions)


In [3]:
import cv2
import numpy as np
from tensorflow.lite.python.interpreter import Interpreter
from PIL import Image
import os

# Define the path to the saved TFLite model
model_path = 'D:/EDI Dataset/trained_model.tflite'

# Load the TFLite model
interpreter = Interpreter(model_path=model_path)
interpreter.allocate_tensors()

# Define a dictionary to map the class indices to their labels
label_map = {0: '10', 1: '20', 2: '50', 3: '100', 4: '200', 5: '500', 6: '2000'}

# Create a unique filename for the captured image
capture_path = 'D:/Captured Images'
if not os.path.exists(capture_path):
    os.makedirs(capture_path)

# Initialize the camera
camera = cv2.VideoCapture(0)

# Set the camera resolution (optional)
camera.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
camera.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

# Define the number of images to capture
num_images = 5

# Capture images and perform inference
for i in range(num_images):
    # Capture an image from the camera
    ret, frame = camera.read()
    
    # Save the captured image
    filename = f'image_{i}.jpg'
    image_path = os.path.join(capture_path, filename)
    cv2.imwrite(image_path, frame)
    
    # Preprocess the image
    img = Image.open(image_path)
    input_tensor = interpreter.get_input_details()[0]['index']
    input_shape = interpreter.get_input_details()[0]['shape']
    resized_image = img.resize((input_shape[1], input_shape[2]))
    input_data = np.array(resized_image).astype('float32')
    input_data = np.expand_dims(input_data, axis=0)
    input_data /= 255.0
    
    # Set the input tensor
    interpreter.set_tensor(input_tensor, input_data)
    
    # Perform inference
    interpreter.invoke()
    
    # Get the output tensor
    output_tensor = interpreter.get_output_details()[0]['index']
    output_data = interpreter.get_tensor(output_tensor)
    
    # Get the predicted class index
    predicted_class_index = np.argmax(output_data)
    
    # Get the predicted class label from the label_map dictionary
    predicted_label = label_map[predicted_class_index]
    
    # Print the predicted label
    print(f'Image: {filename} | Predicted Label: {predicted_label}')
    
# Release the camera
camera.release()


Image: image_0.jpg | Predicted Label: 20
Image: image_1.jpg | Predicted Label: 20
Image: image_2.jpg | Predicted Label: 20
Image: image_3.jpg | Predicted Label: 20
Image: image_4.jpg | Predicted Label: 20
